In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
pio.renderers.default = "browser"

C:\Users\Admin\AppData\Local\Temp/ipykernel_5648/245401626.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Admin\AppData\Local\Temp/ipykernel_5648/245401626.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
covid_conf_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
covid_dead_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
covid_recv_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [4]:
#get data in cleaned time series format for country
#returns dataframe with date and total column
#when you select a country from the dropdown, this function will prepare the time series for that country and return that as dataframe
def process_data(data,cntry='US',window=3):
    conf_ts = data
    conf_ts_cntry = conf_ts[conf_ts['Country/Region']==cntry]
    final_dataset = conf_ts_cntry.T[4:].sum(axis='columns').diff().rolling(window=window).mean()[40:]
    df = pd.DataFrame(final_dataset,columns=['Total'])
    return df
#Here data for the country selected, transposed truncating first 4 rows as they contain some latitude longitude info,
#summing all rows of a country (if there are different rows), diff function gives delta between 2 days, rolling function will help to smoothing the graph
#so this rolling window is used as the slider
#we also need to provide what it needs to do with that rolling window - mean is used so it will give an average of that window

In [5]:
#get overall wordlwide total for confirmed, recovered and dead cases
def get_overall_total(df):
    return df.iloc[:,-1].sum()

conf_overall_total = get_overall_total(covid_conf_ts)
dead_overall_total = get_overall_total(covid_dead_ts)
recv_overall_total = get_overall_total(covid_recv_ts)
print('Overall Confirmed:',conf_overall_total)
print('Overall Dead:',dead_overall_total)
print('Overall Recovered:',recv_overall_total)

Overall Confirmed: 264932878
Overall Dead: 5243348
Overall Recovered: 0


In [6]:
#get total for confirmed, recovered and dead for country (similar to overall total)
def get_cntry_total(df,cntry='US'):
    return df[df['Country/Region']==cntry].iloc[:,-1].sum()

cntry = 'US'
conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
dead_cntry_total = get_cntry_total(covid_dead_ts,cntry)
recv_cntry_total = get_cntry_total(covid_recv_ts,cntry)
print(f'{cntry} Confirmed:',conf_cntry_total)
print(f'{cntry} Dead:',dead_cntry_total)
print(f'{cntry} Recovered:',recv_cntry_total)

US Confirmed: 48990127
US Dead: 787695
US Recovered: 0


In [7]:
#plotly px.line function is used to plot the line chart based on the data frame that is returned from process_data function
#also some minor layout changes have been done like centering the title,and giving the page colour and paper colour
#also naming the axis o the chart
#finally returning as a figure object which will be used by the dash app when we show it on the dash board

def fig_world_trend(cntry='US',window=3):
    df = process_data(data=covid_conf_ts,cntry=cntry,window=window)
    df.head(10)
    if window==1:
        yaxis_title = "Daily Cases"
    else:
        yaxis_title = "Daily Cases ({}-day MA)".format(window)
    fig = px.line(df, y='Total', x=df.index, title='Daily confirmed cases trend for {}'.format(cntry),height=600,color_discrete_sequence =['maroon'])
    fig.update_layout(title_x=0.5,plot_bgcolor='#F2DFCE',paper_bgcolor='#F2DFCE',xaxis_title="Date",yaxis_title=yaxis_title)
    return fig